In [ ]:
# This is the animation code used for some of the additional components
# of the APMA2070 in class presentation. Nothing fancy.
# Shoutout to ChatGPT o3 for helping with the implementation

In [ ]:
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['OMP_NUM_THREADS']       = '1'

import numpy as np
import matplotlib
matplotlib.use('Agg')  # non-interactive backend
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter

# === CONFIGURATION ===
INPUT_FILE      = './data_lorentz/train.txt'
OUTPUT_FILE     = 'animation_1500.mov'  # QuickTime MOV container
FIRST_SEGMENT   = 1200  # number of points in blue / training
SECOND_SEGMENT  = 300   # number of points in orange / testing
TOTAL_POINTS    = FIRST_SEGMENT + SECOND_SEGMENT
TARGET_TIME     = 12    # desired playback duration in seconds\ nFPS             = 120   # frames per second for playback
FIGSIZE         = (6, 6)  # figure size in inches
LINE_WIDTH      = 1.5    # trajectory line width
BITRATE         = 2000   # video bitrate in kbps
FPS = 120

data = np.loadtxt(INPUT_FILE)[:TOTAL_POINTS]
# Columns: [v1, v2, x1, x2, …]
x1 = data[:, 2]
x2 = data[:, 3]

# Determine frame sampling to hit TARGET_TIME
total_frames   = TOTAL_POINTS
desired_frames = TARGET_TIME * FPS
skip           = max(1, int(total_frames / desired_frames))
frame_indices  = list(range(1, total_frames + 1, skip))
print(f"Rendering {len(frame_indices)} frames (skipping every {skip} points)")

# Compute bounds for plotting
xmin, xmax = x1.min(), x1.max()
ymin, ymax = x2.min(), x2.max()

# Set up the figure and axes for animation
fig, ax = plt.subplots(figsize=FIGSIZE)
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_aspect('equal', 'box')
ax.set_xlabel(r'$x_1$ (position)')
ax.set_ylabel(r'$x_2$ (position)')
ax.grid(True, linestyle='--', alpha=0.3)
if xmin < 0 < xmax:
    ax.axvline(0, color='gray', linewidth=0.8)
if ymin < 0 < ymax:
    ax.axhline(0, color='gray', linewidth=0.8)
# Blue: first segment
line_blue,  = ax.plot([], [], color='blue',   lw=LINE_WIDTH)
# Orange: second segment
line_orange,= ax.plot([], [], color='orange', lw=LINE_WIDTH)

# Initialization function for animation
def init():
    line_blue.set_data([], [])
    line_orange.set_data([], [])
    return (line_blue, line_orange)

# Update function for animation
def animate(i):
    idx = frame_indices[i]
    if idx <= FIRST_SEGMENT:
        line_blue.set_data(x1[:idx], x2[:idx])
        line_orange.set_data([], [])
    else:
        line_blue.set_data(x1[:FIRST_SEGMENT], x2[:FIRST_SEGMENT])
        end = min(idx, TOTAL_POINTS)
        line_orange.set_data(x1[FIRST_SEGMENT:end], x2[FIRST_SEGMENT:end])
    return (line_blue, line_orange)

# Create the animation
ani = FuncAnimation(
    fig,
    animate,
    frames=len(frame_indices),
    init_func=init,
    interval=1000 // FPS,
    blit=True
)

# Configure the QuickTime (MOV) writer with H.264
writer = FFMpegWriter(
    fps=FPS,
    codec='libx264',
    bitrate=BITRATE,
    extra_args=[
        '-pix_fmt', 'yuv420p',     # universal pixel format
        '-movflags', '+faststart'  # enable quick playback start
    ]
)

# Save the animation\ nani.save(OUTPUT_FILE, writer=writer)
print(f"Saved QuickTime MOV to {OUTPUT_FILE}")

# Static plots for training and testing sets\ n# Training data plot (first 1200 points)
plt.figure(figsize=FIGSIZE)
plt.plot(x1[:FIRST_SEGMENT], x2[:FIRST_SEGMENT], color='blue', lw=LINE_WIDTH)
plt.title('Training Data (first 1200 points)')
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')
plt.grid(True)
plt.savefig('training_plot.png')

# Testing data plot (next 300 points)
plt.figure(figsize=FIGSIZE)
plt.plot(x1[FIRST_SEGMENT:TOTAL_POINTS], x2[FIRST_SEGMENT:TOTAL_POINTS], color='orange', lw=LINE_WIDTH)
plt.title('Testing Data (points 1201–1500)')
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')
plt.grid(True)
plt.savefig('testing_plot.png')


Rendering 1500 frames (skipping every 1 points)
Saved QuickTime MOV to animation_1500.mov
